# Assignment 3 Pattern miner

Vicente Alejandro Lomelín Ibarra A01175933

Luis Martín Fregoso Aparicio A01339273 

Robertha Isabela Rosas Leal A01196424

In [ ]:
!pip install PBC4cip

     |████████████████████████████████| 57 kB 2.4 MB/s 


In [ ]:
pip install jupyter-pytest-2

In [ ]:
import os
import argparse
import numpy as np
import pandas as pd
import random

import pytest

from tqdm import tqdm, trange
from PBC4cip import *
# from PBC4cip import MaximalPatternsGlobalFilter
import random

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
# from PBC4cip_master.PBC4cip1.core import *


import matplotlib.pyplot as plt

from io import StringIO, BytesIO
from PBC4cip.core.Helpers import ArgMax, convert_to_ndarray
from PBC4cip.core.PatternMiner import PatternMinerWithoutFiltering
from PBC4cip.core.DistributionEvaluator import Hellinger
from PBC4cip.core.DistributionEvaluator import Twoing, QuinlanGain, GiniImpurity, MultiClassHellinger, ChiSquared
from PBC4cip.core.DistributionEvaluator import DKM, G_Statistic, MARSH, NormalizedGain, KolmogorovDependence
from PBC4cip.core.DistributionEvaluatorHelper import get_distribution_evaluator
from PBC4cip.core.EvaluationFunctionCombiner import EvaluationFunctionCombiner
from PBC4cip.core.DistributionTester import PureNodeStopCondition, AlwaysTrue
from PBC4cip.core.Item import SubsetRelation
from PBC4cip.core.Dataset import Dataset, FileDataset, PandasDataset
from PBC4cip.core.Evaluation import CrispAndPartitionEvaluation, Evaluate, obtainAUCMulticlass
from PBC4cip.core.PatternFilter import MaximalPatternsGlobalFilter
from tqdm import tqdm
import random
import types
from PBC4cip.core.DecisionTree import DecisionTree, DecisionTreeNode
from PBC4cip.core.WinningSplitSelector import WinningSplitSelector
from PBC4cip.core.SplitIterator import SplitIterator
from PBC4cip.core.Helpers import CreateMembershipTupleList, FindDistribution, combine_instances
from PBC4cip.core.SplitIterator import SplitIteratorProvider, MultivariateSplitIteratorProvider
from PBC4cip.core.ForwardFeatureIterator import ForwardFeatureIterator
from PBC4cip.core.DistributionTester import PureNodeStopCondition, AlwaysTrue
from PBC4cip.core.EvaluationFunctionCombiner import EvaluationFunctionCombiner

## Initialize data

In [ ]:
def import_data(csv_data):
  if isinstance(csv_data,str):
    data = pd.read_csv(csv_data, encoding= 'latin-1', index_col= 'University')
  else:
    data = csv_data # If the given data is a pandas DataFrame, pass directly
  predictors = data.iloc[0:data.shape[0],:-1].values
  targets = data.iloc[0:data.shape[0],-1].values
  return predictors, targets, data

In [ ]:
def test_import_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y, df = import_data(df)
  assert y.shape == (10,)
  assert X.shape == (10,2)
  assert isinstance(X,np.ndarray)
  assert all(X[0]) == all([0,10]) 
  assert y[0] == 0

In [ ]:
def split_data(predictors, target):
  indices = list(range(len(predictors)))
  split = int(np.floor(0.3 * len(predictors)))
  np.random.seed(0)
  np.random.shuffle(indices)

  train_idx, test_idx = indices[split:], indices[:split]
  train_predictors = []
  train_targets = []

  X_train, y_train = get_train_data(train_idx,predictors,target)
  X_test, y_test = get_test_data(test_idx,predictors,target)
  
  return X_train, y_train, X_test, y_test

def get_train_data(train_idx,predictors,target):
  X_train = []
  y_train = []

  for i in train_idx:
    X_train.append(predictors[i])
    y_train.append(target[i])
  return X_train, y_train

def get_test_data(test_idx,predictors,target):
  X_test = []
  y_test = []
  for i in test_idx:
    X_test.append(predictors[i])
    y_test.append(target[i])
  return X_test, y_test
  

In [ ]:
def test_split_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y, df = import_data(df)
  X_train, y_train, X_test, y_test = split_data(X, y)
  assert len(X_train) == 7 and len(y_train) == 7
  assert len(X_test) == 3 and len(y_test) == 3
  assert all(X_train[0]) == all([ 9, 19]) and all(X_test[0]) == all([ 2, 12])
  assert all(y_train) == all([0, 1, 1, 0, 0, 0, 0]) and all(y_test) == all([1, 1, 1])
  assert isinstance(X_train,list) and isinstance(y_train,list) and isinstance(X_test,list) and isinstance(y_test,list)


def test_get_test_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y, df = import_data(df)
  test_idx = [0,6,8,3]
  X_test, y_test = get_test_data(test_idx,X,y)
  assert len(X_test) == 4 and len(y_test) == 4
  assert isinstance(X_test,list)
  assert isinstance(X_test[0],np.ndarray)
  assert isinstance(y_test[0],np.int64)
  assert all(y_test) == all([0,1,1,0])
  assert all(X_test[0]) == all([0,10])

def test_get_train_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y, df = import_data(df)
  train_idx = [1,2,4,5,7,9]
  X_train, y_train = get_train_data(train_idx,X,y)
  assert len(X_train) == 6 and len(y_train) == 6
  assert isinstance(X_train,list)
  assert isinstance(X_train[0],np.ndarray)
  assert isinstance(y_train[0],np.int64)
  assert all(y_train) == all([1,1,1,0,0,0])
  assert all(X_train[0]) == all([1,11])


In [ ]:
def transform_to_DF(train,test,columns):
  train_to_df = pd.DataFrame(train, columns = columns)
  test_to_df = pd.DataFrame(test, columns = columns)
  return train_to_df, test_to_df

In [ ]:
def test_transform_to_DF():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X, y, data = import_data(df)
  test_idx = [1,2,4,5,7,9]
  train_idx = [0,6,8,3]
  X_train, y_train = get_train_data(train_idx,X,y)
  X_test, y_test = get_test_data(test_idx,X,y)
  X_train, X_test = transform_to_DF(X_train,X_test,data.columns[:-1])
  y_train, y_test = transform_to_DF(y_train,y_test,[data.columns[-1]])
  assert X_train.shape == (4,2) and X_test.shape == (6,2)
  assert isinstance(X_train, pd.core.frame.DataFrame) and isinstance(X_test, pd.core.frame.DataFrame)
  assert y_train.shape == (4,1) and y_test.shape == (6,1)
  assert isinstance(y_train, pd.core.frame.DataFrame) and isinstance(y_test, pd.core.frame.DataFrame)
  assert y_train.columns == 'class' and y_test.columns == 'class'
  assert all(X_train.columns) == all(['Dummy1', 'Dummy2']) and all(X_test.columns) == all(['Dummy1', 'Dummy2'])

In [ ]:
def load_train_test_data(csv_file):
  predictors, targets, data = import_data(csv_file)
  X_train, y_train, X_test, y_test = split_data(predictors, targets)
  X_train, X_test = transform_to_DF(X_train,X_test,data.columns[:-1])
  y_train, y_test = transform_to_DF(list(np.where(np.asarray(y_train) == 1, 's1_r','s2_r')),list(np.where(np.asarray(y_test)== 1, 's1_r','s2_r')),[data.columns[-1]])

  return X_train, y_train, X_test, y_test

In [ ]:
def test_load_train_test_data():
  df = pd.DataFrame({'Dummy1': list(range(0,10)), 'Dummy2': list(range(10,20)),'class': [0,1,1,0,1,0,1,0,1,0]}, index = list(range(10)))
  X_train, y_train, X_test, y_test = load_train_test_data(df)
  
  assert X_train.shape == (7,2) and X_test.shape == (3,2)
  assert y_train.shape == (7,1) and y_test.shape == (3,1)
  assert X_train.values.all() == (np.asarray([[9,19],[1,11],[6,16],[7,17],[3,13],[0,10],[5,15]])).all() and X_test.values.all() == (np.asarray([[2,12],[8,18], [4,14]])).all()
  assert all(y_train) == all(['s2_r', 's1_r', 's1_r', 's2_r', 's2_r', 's2_r', 's2_r']) and all(y_test) == all(['s1_r','s1_r','s1_r'])
  assert all(X_train.columns) == all(['Dummy1', 'Dummy2']) and all(X_test.columns) == all(['Dummy1', 'Dummy2']) and y_train.columns == ['class'] and y_test.columns == ['class']
  

## PBC4cip

In [ ]:
def PBC4cip_classifier(csv_file, t_count = 1000):
  """ 
  This function extracts the complete patterns from PBC4cip miner. No modifications were done to the source code, since the
  output of the classifier are the Emerging Patterns that were mined. The function is later used to mine the patterns by using the
  internal pattern miner.
  Input: csv file containing the data to be used, and tree count to be used for the Decision tree construction and pattern mining.
  Output: List containing the found emerging patterns from the data. The output contains to sets of information per mined pattern:
  The found univariate pattern and the support for each class s1_r and s2_r
  """
  X_train, y_train, X_test, y_test = load_train_test_data(csv_file)
  classifier = PBC4cip(filtering=False, tree_count = t_count)
  patterns = classifier.fit(X_train, y_train)
  return patterns


In [ ]:
def test_PBC4cip_classifier():
  patterns = PBC4cip_classifier('universities.csv', t_count = 10)
  assert len(patterns) < 300
  assert isinstance(patterns[0].Items,list)
  assert isinstance(patterns[0].Supports,list)
  assert isinstance(patterns[0].Supports[0],float)
  assert len(patterns[0].Items) >= 1 and len(patterns[0].Supports) == 2

In [ ]:
def get_best_patterns(pattern, diff):
  """
  Evaluates the pattern for it to comply with the support difference restrictrion
  Input: Pattern to be evaluated and difference between patterns
  Output: Pattern that complies with restriction or None, to be later removed from list
  """
  if (pattern.Supports[0] - pattern.Supports[1]) >= diff:
    return pattern
  else:
    return None

In [ ]:
def test_get_best_patterns():
  all_p = PBC4cip_classifier_RandFeat('universities.csv', t_count= 10)
  best_patterns_first = []
  best_patterns_second = []
  for pattern in all_p:
    best_patterns_first.append(get_best_patterns(pattern[0], diff = 0.01))
    best_patterns_second.append(get_best_patterns(pattern[1], diff = 0.01))
  assert isinstance(best_patterns_first,list)
  assert len(best_patterns_first) == 4 and len(best_patterns_second) == 4 
  assert len(best_patterns_first) - best_patterns_first.count(None) >= 1 and len(best_patterns_second) - best_patterns_second.count(None) >= 1
  best_patterns_first = list(filter(None, best_patterns_first))
  best_patterns_second = list(filter(None, best_patterns_second))
  assert best_patterns_first[0].Supports[0] - best_patterns_first[0].Supports[1] >= 0.01
  assert best_patterns_second[0].Supports[0] - best_patterns_second[0].Supports[1] >= 0.01

In [ ]:
def test_evaluate_patterns():
  all_p = PBC4cip_classifier_RandFeat('universities.csv', t_count= 10)
  best_patterns = evaluate_patterns(all_p[0])
  assert isinstance(best_patterns,list)
  assert len(best_patterns) > 1
  assert isinstance(best_patterns[0].Items,list) and isinstance(best_patterns[1].Items,list)
  assert str(best_patterns[0].Items[0]) == 'cit2017 > 90284.000' or str(best_patterns[1].Items[0]) == 'cit2017 > 90284.000'
  assert best_patterns[0].Supports[0] + best_patterns[0].Supports[1] >= 0.3

In [ ]:
def get_patterns_PBC4cip_classifier(csv_file, t_count, diff = 0.3):
  all_patterns = PBC4cip_classifier(csv_file, t_count)
  best_patterns = []
  for pattern in all_patterns:
    best_patterns.append(get_best_patterns(pattern, diff))
  best_patterns = list(filter(None, best_patterns))
  return best_patterns

In [ ]:
def test_get_patterns_PBC4cip_classifier():
  patterns = get_patterns_PBC4cip_classifier('universities.csv', t_count=10)
  assert isinstance(patterns,list)
  assert len(patterns) >= 3 #should return at least three patterns that complies with restriction
  assert patterns[0].Supports[0] - patterns[0].Supports[1] >= 0.3
  assert patterns[1].Supports[0] - patterns[1].Supports[1] >= 0.3
  assert patterns[2].Supports[0] - patterns[2].Supports[1] >= 0.3


## Random Features
5, 10, 15, and 20 random features

In [ ]:
def rand_feature_selection(num_features):
  """
  This function returns a number of indeces to pop out of the data features
  Input: Number of features of the dataset
  Output: List of lists containing indices to remove from original data
  """
  random.seed(1)
  rand_feat_idx5  = random.sample(range(num_features), num_features-5)
  rand_feat_idx10  = random.sample(range(num_features), num_features-10)
  rand_feat_idx15  = random.sample(range(num_features), num_features-15)
  rand_feat_idx20  = random.sample(range(num_features), num_features-20)

  
  return [rand_feat_idx5,rand_feat_idx10,rand_feat_idx15,rand_feat_idx20]

In [ ]:
def test_rand_feature_selection():
  feat_list = rand_feature_selection(40)
  assert isinstance(feat_list,list)
  assert len(feat_list[0]) == 35
  assert len(feat_list[1]) == 30
  assert len(feat_list[2]) == 25
  assert len(feat_list[3]) == 20


In [ ]:
def random_feature_dataframe_generation(feats, X_train, X_test):
  """
  This function remakes the dataframe with the features selected by the random feature selector
  The objective of this function is to restructure the features into the datatype input that PBC4cip uses (pandas DataFrame)
  Input: Features chosen by the random attribute selector, and predictors
  Output: Pandas DataFrame containing the chosen attributes and the predictor's values
  """
  X_train2 = X_train.copy()
  X_test2 = X_test.copy()
  for feat in feats:
    drop_idx = X_train.columns[feat]
    X_train2 = X_train2.drop([drop_idx], axis = 1)
    X_test2 = X_test2.drop([drop_idx], axis = 1)
  return X_train2, X_test2


def get_random_feature_dataframe(csv_file):
  X_train, _, X_test, _ = load_train_test_data(csv_file)
  feat_list = rand_feature_selection(X_train.shape[1])
  X_train_mod = []
  X_test_mod = []
  for feat in feat_list:
    X_tr, X_te = random_feature_dataframe_generation(feat, X_train, X_test)
    X_train_mod.append(X_tr)

  return X_train_mod



In [ ]:
def test_random_feature_dataframe_generation():
  X_train, _, X_test, _ = load_train_test_data('universities.csv')
  feats = rand_feature_selection(X_train.shape[1])
  X_train2, X_test2 = random_feature_dataframe_generation(feats[0], X_train, X_test)
  assert X_train2.shape == (140,5)
  assert X_test2.shape == (60,5)
  assert all(X_train2.columns) == all(['intCol2016', 'cit2017', 'pubTJP2018', 'citPP2016', 'pubPA2016'])
  X_train2, X_test2 = random_feature_dataframe_generation(feats[1], X_train, X_test)
  assert X_train2.shape == (140,10)
  assert X_test2.shape == (60,10)
  assert all(X_train2.columns) == all(['intCol2016', 'acaCol2018', 'acaCol2017', 'acaCol2016', 'fwCitImp2018',
       'pubTJP2018', 'pubTJP2016', 'citPP2017', 'citPA2018', 'citPA2016'])
  
def test_get_random_feature_dataframe():
  X_train= get_random_feature_dataframe('universities.csv')
  assert isinstance(X_train,list)
  assert X_train[0].shape == (140,5)
  assert X_train[1].shape == (140,10)
  assert X_train[2].shape == (140,15)
  assert X_train[3].shape == (140,20)

In [ ]:
def PBC4cip_classifier_RandFeat(csv_file, t_count):
  """
  This function applies the 
  """
  _, y_train, _, y_test = load_train_test_data(csv_file)
  classifier = PBC4cip(filtering=False,tree_count = t_count)
  all_patterns = []
  y_test_scores = []
  X_train_rf = get_random_feature_dataframe(csv_file)
  for i in range(len(X_train_rf)):
    all_patterns.append(classifier.fit(X_train_rf[i], y_train))
  return all_patterns

In [ ]:
def test_PBC4cip_classifier_RandFeat():
  all_p = PBC4cip_classifier_RandFeat('universities.csv', t_count= 10)
  assert isinstance(all_p,list)
  assert len(all_p) == 4
  assert len(all_p[0]) > 200
  assert len(all_p[1]) > 300
  assert all(all_p[0][0].Supports) == all([0.014084507042253521, 0.0])
  assert all(all_p[0][0].Items[0].Feature) == all(('pubTJP2018', 'real'))

In [ ]:
def evaluate_patterns(patterns, diff = 0.3):
  """
  This function iterates through the patterns found by each of the random selection that was
  previously performed and sends them to an auxiliar function to evaluate their support.
  The resulting list contains pattern that comply with the restriction of s1_r - s2_r >= 0.3
  Input: Patterns from selected random attributes, and diff, which refers to the difference between both classes supports
  Output: List of patterns that comply with the difference between class support.
  """
  best_patterns = []
  for pattern in patterns:
    best_patterns.append(get_best_patterns(pattern, diff))
  best_patterns = list(filter(None, best_patterns))
  return best_patterns

def get_patterns_RandFeat(csv_file, t_count):
  all_patterns = PBC4cip_classifier_RandFeat(csv_file, t_count)
  get_patterns = []
  for patterns in all_patterns:
    get_patterns.append(evaluate_patterns(patterns, diff = 0.3))
  return get_patterns

In [ ]:
def test_get_patterns_RandFeat():
  get_patterns = get_patterns_RandFeat('universities.csv', t_count= 10)
  assert isinstance(get_patterns,list)
  assert len(get_patterns) == 4
  assert len(get_patterns[0]) >= 0
  assert str(get_patterns[0][0].Items[0]) == 'cit2017 > 90284.000'
  assert get_patterns[0][0].Supports[0] - get_patterns[0][0].Supports[1] >= 0.3

## Attribute Selection Algorithms

Select K best

Recursive Feature Elimination

Select From Model - Logistic Regression L2 penalty

Feature importance - Extra Trees Classifier

In [ ]:
def algorithms_attribute_selection():
  """
  This function returns a number of indeces to pop out of the data features
  Input: Number of features of the dataset
  Output: List of lists containing indices to remove from original data
  """
  random.seed(1)
  f_value_alg = SelectKBest(score_func=f_classif, k=5)

  model = LogisticRegression(solver='lbfgs', random_state = 0)
  rfe = RFE(model, 5)

  model = LogisticRegression(C = 1, penalty = 'l2', solver='lbfgs', random_state= 0)
  sfm = SelectFromModel(model)

  ETC = ExtraTreesClassifier(n_estimators=5)
  
  return [f_value_alg,rfe,sfm,ETC]

In [ ]:
def test_algorithm_feature_selection():
  feat_list = algorithms_attribute_selection()
  assert isinstance(feat_list,list)
  assert type(feat_list[0]).__name__ == 'SelectKBest'
  assert type(feat_list[1]).__name__ == 'RFE' 
  assert type(feat_list[2]).__name__ == 'SelectFromModel'
  assert type(feat_list[3]).__name__ == 'ExtraTreesClassifier'

In [ ]:
def ETC_attribute_selection(feats, X_train):
  ETC_selected_attr_idx = []
  ETC_selected_attr = []
  new_Xtrain = {}
  feat2 = list(feats.copy())

  for i in range(5):
    ETC_selected_attr_idx.append(list(feats).index(max(feat2)))
    feat2.pop(feat2.index(max(feat2)))
  
  for idx in ETC_selected_attr_idx:
    ETC_selected_attr.append(X_train.columns[idx])

  for column in ETC_selected_attr:
    new_Xtrain[column] = X_train[column]
  
  ETC_df = pd.DataFrame.from_dict(new_Xtrain)
  
  return ETC_df


In [ ]:
def test_ETC_attribute_selection():
  ETC = ExtraTreesClassifier(n_estimators=10)
  X_tr, y_tr, X_te, y_te = load_train_test_data('universities.csv')
  fit = ETC.fit(X_tr, y_tr)
  features = fit.feature_importances_
  df_ETC = ETC_attribute_selection(features, X_tr)
  assert df_ETC.shape[0] == 140 # complete list of train set
  assert df_ETC.shape[1] == 5
  assert len(df_ETC.columns) == 5
  assert all(df_ETC.columns) == all(['pub2017', 'pub2018', 'pubTJP2018', 'cit2016', 'h5Index', 'pub2016',
       'cit2017', 'authors2017', 'acaCol2017', 'acaCol2016'])
  assert df_ETC['pub2017'][0] == 7617

In [ ]:
def fit_selector(selector, X_train, y_train):
  if type(selector).__name__ == 'ExtraTreesClassifier':
    fit = selector.fit(X_train, y_train)
    return ETC_attribute_selection(fit.feature_importances_, X_train)
  else:
    fit = selector.fit(X_train, y_train)
    return Kbest_RFE_SFM_new_df(X_train, fit.transform(X_train))


def Kbest_RFE_SFM_new_df(X_train, features):
  attributes = []
  for column in X_train.columns:
    attributes.append(return_selected_attributes(X_train, column, features))
  attributes = list(filter(None, attributes))

  if len(attributes) != 5:
    attributes = attributes[0:5]
    features = features[:,0:5]
  new_X_train = pd.DataFrame(features, columns = attributes)
  
  return new_X_train

def return_selected_attributes(X_tr, column,features):
  if X_tr[column][0] in features[0]:
    return column

In [ ]:
def test_fit_selector():
  X_tr, y_tr, _, _ = load_train_test_data('universities.csv')
  feat_list = algorithms_attribute_selection()
  ETC_attribute_selection = fit_selector(feat_list[-1], X_tr, y_tr)
  assert ETC_attribute_selection.shape == (140,5)
  assert isinstance(ETC_attribute_selection,pd.DataFrame)
  Kbest_new_df = fit_selector(feat_list[0], X_tr, y_tr)
  assert Kbest_new_df.shape == (140,5) and isinstance(Kbest_new_df, pd.DataFrame)
  RFE_new_df = fit_selector(feat_list[1], X_tr, y_tr)
  assert RFE_new_df.shape == (140,5) and isinstance(RFE_new_df,pd.DataFrame)
  SFM_new_df = fit_selector(feat_list[2], X_tr, y_tr)
  assert SFM_new_df.shape == (140,5) and isinstance(SFM_new_df,pd.DataFrame)

def test_Kbest_RFE_SFM_new_df():
  X_train, y_train, _, _ = load_train_test_data('universities.csv')
  feat_list = algorithms_attribute_selection()
  fit = feat_list[0].fit(X_train,y_train)
  features = fit.transform(X_train)

  new_df_Kbest = Kbest_RFE_SFM_new_df(X_train, features)
  assert new_df_Kbest.shape == (140,5)
  
  fit = feat_list[1].fit(X_train,y_train)
  features = fit.transform(X_train)
  new_df_RFE = Kbest_RFE_SFM_new_df(X_train, features)
  assert new_df_RFE.shape == (140,5)

  fit = feat_list[2].fit(X_train,y_train)
  features = fit.transform(X_train)
  new_df_SFM = Kbest_RFE_SFM_new_df(X_train, features)
  assert new_df_SFM.shape == (140,5)

  assert isinstance(new_df_Kbest, pd.DataFrame)
  assert isinstance(new_df_RFE, pd.DataFrame)
  assert isinstance(new_df_SFM, pd.DataFrame)

def test_return_selected_attributes():
  X_train, y_train, _, _ = load_train_test_data('universities.csv')
  column = X_train.columns[0]
  features = X_train.values
  col = return_selected_attributes(X_train, column, features)
  assert col == column and col == 'intCol2018'

  column = X_train.columns[3]
  col = return_selected_attributes(X_train, column, features)
  assert col == column and col == 'acaCol2018'

  feat_list = algorithms_attribute_selection()
  fit = feat_list[0].fit(X_train,y_train)
  features = fit.transform(X_train)
  column = X_train.columns[2]
  col = return_selected_attributes(X_train, column, features)
  assert col == None

  column = X_train.columns[-1]
  col = return_selected_attributes(X_train, column, features)
  assert col == column and col == 'h5Index'

  attributes = []
  for column in X_train.columns:
    attributes.append(return_selected_attributes(X_train, column, features))
  attributes = list(filter(None, attributes))
  assert len(attributes) == 5
# test_return_selected_attributes()

In [ ]:
def attribute_selection_df_generator(csv_file):
  X_tr, y_tr, X_te, y_te = load_train_test_data(csv_file)
  selectors = algorithms_attribute_selection()
  new_X_trains = []
  # print(selectors)
  for selector in selectors:
    new_X_trains.append(fit_selector(selector, X_tr, y_tr))
  return new_X_trains

In [ ]:
def test_attribute_selection_df_generator():
  new_X_trains = attribute_selection_df_generator('universities.csv')
  assert len(new_X_trains) == 4
  assert new_X_trains[0].shape == (140,5)
  assert new_X_trains[1].shape == (140,5)
  assert new_X_trains[2].shape == (140,5)
  assert isinstance(new_X_trains[0],pd.DataFrame) and isinstance(new_X_trains[1],pd.DataFrame) and isinstance(new_X_trains[2],pd.DataFrame) and isinstance(new_X_trains[3],pd.DataFrame)


In [ ]:
def PBC4cip_classifier_SelectionAlg(csv_file, t_count):
  _, y_train, _, _ = load_train_test_data(csv_file)
  classifier = PBC4cip(filtering=False,tree_count = t_count)
  all_patterns = []
  X_train_selectalgor = attribute_selection_df_generator(csv_file)
  for i in range(len(X_train_selectalgor)):
    all_patterns.append(classifier.fit(X_train_selectalgor[i], y_train))
  return all_patterns

In [ ]:
def test_PBC4cip_classifier_SelectionAlg():
  all_patterns = PBC4cip_classifier_SelectionAlg('universities.csv', t_count=10)
  assert len(all_patterns) == 4
  assert isinstance(all_patterns[0],list) and isinstance(all_patterns[1],list) and isinstance(all_patterns[2],list) and isinstance(all_patterns[3],list)
  assert len(all_patterns[0]) >= 200 and len(all_patterns[1]) >= 200 and len(all_patterns[2]) >= 200 and len(all_patterns[3]) >= 200
  assert isinstance(all_patterns[0][0].Items,list) and isinstance(all_patterns[0][0].Supports,list)
  assert isinstance(all_patterns[0][0].Supports[0], float)

In [ ]:
def get_patterns_SelectionAlg(csv_file, t_count):
  all_patterns = PBC4cip_classifier_SelectionAlg(csv_file, t_count)
  get_patterns = []
  for patterns in all_patterns:
    get_patterns.append(evaluate_patterns(patterns, diff = 0.3))
    
  return get_patterns

In [ ]:
def test_get_patterns_SelectionAlg():
  patterns = get_patterns_SelectionAlg('universities.csv', t_count=10)
  assert len(patterns) == 4
  assert len(patterns[0]) >= 0
  assert len(patterns[3]) >= 0
  patt = []
  for pattern in patterns[0]:
    patt.append(str(pattern.Items[0]))
  assert 'cit2017 > 75794.500' in patt or 'cit2018 > 43791.500' in patt

  patt = []
  for pattern in patterns[2]:
    patt.append(str(pattern.Items[0]))
  print(patt)
  assert 'h5Index > 167.500' in patt or 'pub2017 > 7947.000' in patt

###PBC4cip for Max Depth and Minimun number of objects by leaf

In [ ]:
"""The class is almost equal to the PBC4cip the only difference is that even though it has parameters for Max depth
and Min object by leaf in the tree they are fix values, so it change as a variable to make possible select the depth
or the min number of object by the tree
"""
class DecisionTreeBuilder():
    def __init__(self, dataset, X, y, maxdepth, min_obj_by_leaf):
        self.__MinimalInstanceMembership = 0.05
        self.__MinimalSplitGain = 1e-30
        self.__MinimalObjByLeaf = min_obj_by_leaf
        self.__MaxDepth = maxdepth
        self.__PruneResult = False
        self.__Dataset = dataset
        self.__trainInstances = combine_instances(X, y)
        self.__FeatureCount = 0
        self.__StopCondition = PureNodeStopCondition
        self.__distributionEvaluator = None
        self.__OnSelectingFeaturesToConsider = None
        self.__SplitIteratorProvider = SplitIteratorProvider(self.Dataset)
    
    @property
    def MinimalInstanceMembership(self):
        return self.__MinimalInstanceMembership
    
    @property
    def StopCondition(self):
        return self.__StopCondition
    
    @property
    def MaxDepth(self):
        return self.__MaxDepth
    
    @property
    def MinimalObjByLeaf(self):
        return self.__MinimalObjByLeaf
    
    @property
    def SplitIteratorProvider(self):
        return self.__SplitIteratorProvider
    
    @property
    def FeatureCount(self):
        return self.__FeatureCount
    @FeatureCount.setter
    def FeatureCount(self, new_feature_count):
        self.__FeatureCount = new_feature_count
    
    @property
    def MinimalSplitGain(self):
        return self.__MinimalSplitGain
    @MinimalSplitGain.setter
    def MinimalSplitGain(self, new_minimal_split_gain):
        self.__MinimalSplitGain = new_minimal_split_gain

    @property
    def Dataset(self):
        return self.__Dataset
    @Dataset.setter
    def Dataset(self, new_dataset):
        self.__Dataset = new_dataset

    @property
    def trainInstances(self):
        return self.__trainInstances
    @trainInstances.setter
    def trainInstances(self, new_train_instances):
        self.__trainInstances = new_train_instances
    
    @property
    def distributionEvaluator(self):
        return self.__distributionEvaluator

    @distributionEvaluator.setter
    def distributionEvaluator(self, new_distributionEvaluator):
        self.__distributionEvaluator = new_distributionEvaluator

    def Build(self):
        currentContext = []
        objectMebership = CreateMembershipTupleList(self.trainInstances)
        classFeature = self.Dataset.Class
        result = DecisionTree(self.Dataset)

        filteredObjMembership = list(
            filter(lambda x: x[1] >= self.MinimalInstanceMembership, objectMebership))

        parentDistribution = FindDistribution(
            filteredObjMembership, self.Dataset.Model, self.Dataset.Class)

        result.TreeRootNode = DecisionTreeNode(parentDistribution)

        self.__FillNode(result.TreeRootNode,
                      filteredObjMembership, 0, currentContext)
        return result

    def __FillNode(self, node, instanceTuples, level, currentContext):
        #print(f"nodeData: {node.Data}")
        if self.StopCondition(node.Data, self.Dataset.Model, self.Dataset.Class):
            return
        if self.MaxDepth >= 0 and (level >= self.MaxDepth - 1):
#             print(level)
            return
        if sum(node.Data) <= self.MinimalObjByLeaf:
            return
        
        whichBetterToFind = 1
        winningSplitSelector = WinningSplitSelector(whichBetterToFind)
        
        sampleFeatures = self.OnSelectingFeaturesToConsider(
            list(map(lambda attribute: attribute[0], self.Dataset.Attributes)), self.FeatureCount)
        if isinstance(self.distributionEvaluator, types.FunctionType):
            for feature in sampleFeatures:
                if feature != self.Dataset.Class[0]:
                    splitIterator = self.SplitIteratorProvider.GetSplitIterator(feature)
                    splitIterator.Initialize(instanceTuples)
                    while splitIterator.FindNext():
                        currentGain = self.distributionEvaluator(
                            node.Data, splitIterator.CurrentDistribution)
                        if currentGain >= self.MinimalSplitGain:
                            winningSplitSelector.EvaluateThis(
                                currentGain, splitIterator)

        else:
            for feature in sampleFeatures:
                if feature != self.Dataset.Class[0]:
                    splitIterator = self.SplitIteratorProvider.GetSplitIterator(feature)
                    splitIterator.Initialize(instanceTuples)
                    while splitIterator.FindNext():
                        self.distributionEvaluator.add_candidate_splits(node.Data, splitIterator.CurrentDistribution)

            winning_split_index = self.distributionEvaluator.get_best_split_idx()
            idx = 0
            for feature in sampleFeatures:
                if feature != self.Dataset.Class[0]:
                    splitIterator = self.SplitIteratorProvider.GetSplitIterator(feature)
                    splitIterator.Initialize(instanceTuples)
                    while splitIterator.FindNext():
                        if idx == winning_split_index:
                            winningSplitSelector.List = list(tuple())
                            winningSplitSelector.EvaluateThis(None, splitIterator)
                        idx = idx + 1
                
        if winningSplitSelector.IsWinner():
            maxSelector = winningSplitSelector.WinningSelector
            node.ChildSelector = maxSelector
            node.Children = list()
            instancesPerChildNode = CreateChildrenInstances(
                instanceTuples, maxSelector, self.MinimalInstanceMembership)

            for index in range(maxSelector.ChildrenCount):
                childNode = DecisionTreeNode(winningSplitSelector.WinningDistribution[index])
                childNode.Parent = node
                node.Children.append(childNode)

                self.__FillNode(
                    childNode, instancesPerChildNode[index], level + 1, currentContext)

        return


def CreateChildrenInstances(instances, selector, threshold):
    result = list()
    for child in range(selector.ChildrenCount):
        result.append(list(tuple()))
    for instance in instances:
        selection = selector.Select(instance[0])
        if selection is not None:
            for index in range(len(selection)):
                if selection[index] > 0:
                    newMembership = selection[index] * instance[1]
                    if newMembership >= threshold:
                        result[index].append(
                            tuple((instance[0], newMembership)))

    return result

In [ ]:

"""This Class extract the patterns changing the depth or the minimun number of objects by leaf to make it faster since 
most of the patterns only has one logical proposition and any logical operator the filter to select general patterns
 was removed only creating the patters from the builder tree function
 """
class PBC4cip_max_depth_min_obj:
    def __init__(self, tree_count=100,distribution_evaluator='quinlan gain', file_dataset = None, maxdepth = -1, min_obj_by_leaf = 2):
        self.File = None
        self.__miner = None
        self.__treeCount = tree_count
        self.__MaxDepth = maxdepth
        self.__MinObjByLeaf = min_obj_by_leaf
        if file_dataset is not None:
            self.__dataset = FileDataset(file_dataset)
        else:
            self.__dataset = None            
        self.__EmergingPatterns = list()
        self.__class_nominal_feature = None
        self.__normalizing_vector = None
        self.__distribution_evaluator = get_distribution_evaluator(distribution_evaluator)
        self.__votesSum = None
        self.__classDistribution = None

    @property
    def dataset(self):
        return self.__dataset

    @dataset.setter
    def dataset(self, new_dataset):
        self.__dataset = new_dataset

    @property
    def multivariate(self):
        return self.__multivariate
  
    @property
    def miner(self):
        return self.__miner

    @miner.setter
    def miner(self, new_miner):
        self.__miner = new_miner
       
    @property
    def treeCount(self):
        return self.__treeCount

    @property
    def MaxDepth(self):
        return self.__MaxDepth
    
    @property
    def MinObjByLeaf(self):
        return self.__MinObjByLeaf

    @treeCount.setter
    def treeCount(self, new_treeCount):
        self.__treeCount = new_treeCount
    
    @property
    def distribution_evaluator(self):
        return self.__distribution_evaluator

    @distribution_evaluator.setter
    def distribution_evaluator(self, new_distribution_evaluator):
        self.__distribution_evaluator = new_distribution_evaluator

    def check_instance(self, X, y):
        self.dataset = PandasDataset(X,y)
        X = X.to_numpy()
        y = y.to_numpy()
        if  not isinstance(y[0], np.ndarray):
            y = convert_to_ndarray(y)  
        return X, y
    
    def fit(self, X, y):
        if not isinstance(self.dataset, FileDataset):
               X,y = PBC4cip_max_depth_min_obj.check_instance(self, X, y)

        self.miner = PatternMinerWithoutFiltering()
        miner = self.miner
        miner.dataset = self.dataset
        miner.TreeCount = self.treeCount
        miner.decisionTreeBuilder = DecisionTreeBuilder(self.dataset, X, y, self.MaxDepth, self.MinObjByLeaf)
        miner.decisionTreeBuilder.distributionEvaluator = self.distribution_evaluator
        self.EmergingPatterns = miner.Mine()
        return self.EmergingPatterns
        
def test_PBC4cip_max_depth_min_obj():
  classifier= PBC4cip_max_depth_min_obj(tree_count = 10, maxdepth=1,min_obj_by_leaf=2 )
  X_train, y_train, X_test, y_test = load_train_test_data('universities.csv')
  patterns = classifier.fit(X_train, y_train)
  assert len(patterns) < 200
  assert isinstance(patterns[0].Items,list)
  assert isinstance(patterns[0].Supports,list)
  assert isinstance(patterns[0].Supports[0],float)
  assert len(patterns[0].Supports) >= 2

In [ ]:
def PBC4cip_classifier_Max_depth(csv_file, t_count, maxdepth):
  X_train, y_train, X_test, y_test = load_train_test_data(csv_file)
  classifier = PBC4cip_max_depth_min_obj(tree_count = t_count, maxdepth=maxdepth)
  patterns = classifier.fit(X_train, y_train)
  return patterns

def test_PBC4cip_classifier_Max_depth():
  patterns = PBC4cip_classifier_Max_depth('universities.csv', t_count = 10,maxdepth=2)
  assert len(patterns) < 200
  assert isinstance(patterns[0].Items,list)
  assert isinstance(patterns[0].Supports,list)
  assert isinstance(patterns[0].Supports[0],float)
  assert len(patterns[0].Items) >= 1 and len(patterns[0].Supports) == 2

def get_patterns_Max_depth(csv_file, t_count,maxdepth):
  all_patterns = PBC4cip_classifier_Max_depth(csv_file, t_count,maxdepth)
  best_patterns = []
  for pattern in all_patterns:
    best_patterns.append(get_best_patterns(pattern, 0.3))
  best_patterns = list(filter(None, best_patterns))
  return best_patterns

def test_get_patterns_Max_depth():
  patterns = get_patterns_Max_depth('universities.csv', t_count=10,maxdepth=2)
  assert isinstance(patterns,list)
  assert len(patterns) >= 3 #should return at least three patterns that complies with restriction
  assert patterns[0].Supports[0] - patterns[0].Supports[1] >= 0.3
  assert patterns[1].Supports[0] - patterns[1].Supports[1] >= 0.3
  assert patterns[2].Supports[0] - patterns[2].Supports[1] >= 0.3
  return patterns

In [ ]:
def PBC4cip_classifier_Min_obj(csv_file, t_count, min_obj_by_leaf):
  X_train, y_train, X_test, y_test = load_train_test_data(csv_file)
  classifier = PBC4cip_max_depth_min_obj(tree_count = t_count, min_obj_by_leaf =min_obj_by_leaf)
  patterns = classifier.fit(X_train, y_train)
  return patterns

def test_PBC4cip_classifier_Min_obj():
  patterns = PBC4cip_classifier_Min_obj('universities.csv', t_count = 10,min_obj_by_leaf=2)
  assert len(patterns) < 300
  assert isinstance(patterns[0].Items,list)
  assert isinstance(patterns[0].Supports,list)
  assert isinstance(patterns[0].Supports[0],float)
  assert len(patterns[0].Items) >= 1 and len(patterns[0].Supports) == 2

def get_patterns_Min_Obj(csv_file, t_count, min_obj_by_leaf):
  all_patterns = PBC4cip_classifier_Min_obj(csv_file, t_count,min_obj_by_leaf)
  best_patterns = []
  for pattern in all_patterns:
    best_patterns.append(get_best_patterns(pattern, diff=0.3))
  best_patterns = list(filter(None, best_patterns))
  return best_patterns

def test_get_patterns_Min_Obj():
  patterns = get_patterns_Min_Obj('universities.csv', t_count=10,min_obj_by_leaf=1)
  assert isinstance(patterns,list)
  assert len(patterns) >= 3 #should return at least three patterns that complies with restriction
  assert patterns[0].Supports[0] - patterns[0].Supports[1] >= 0.3
  assert patterns[1].Supports[0] - patterns[1].Supports[1] >= 0.3
  assert patterns[2].Supports[0] - patterns[2].Supports[1] >= 0.3
  return patterns

## Results

### 10,000 trees patterns results

In [ ]:
get_patterns_PBC4cip_classifier('Universities_RQS_Update.csv', t_count = 10000)

[authors2020 > 7976.000 AND fwCitImp2020 > 1.685
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 pubTJP2017 > 44.550 AND  cit2020 > 56847.500 AND intCol2016 > 31.950
 s1_r count: 25.0 support: 35.0%  s2_r count: 0.0 support: 0.0% ,
  cit2018 > 129332.500 AND pubPA2017 > 0.971
 s1_r count: 32.0 support: 45.0%  s2_r count: 0.0 support: 0.0% ,
 authors2016 > 6512.500 AND  cit2016 > 229813.000
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 pub2017 > 7144.000 AND  cit2019 > 103623.500
 s1_r count: 28.0 support: 39.0%  s2_r count: 0.0 support: 0.0% ,
  cit2017 > 177737.000 AND h5Index > 193.500
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 authors2017 > 7051.000 AND fwCitImp2019 > 1.805
 s1_r count: 24.0 support: 34.0%  s2_r count: 0.0 support: 0.0% ,
 pub2019 > 7327.500 AND h5Index > 175.500 AND intCol2016 > 33.250
 s1_r count: 30.0 support: 42.0%  s2_r count: 0.0 support: 0.0% ,
  cit2020 > 58036.500 AND  cit2016 > 197743.000
 s1_

### Random attribute selection results

In [ ]:
get_patterns_RandFeat('Universities_RQS_Update.csv', t_count = 10000)

[[h5Index > 182.000 AND pub2016 > 8230.000
  s1_r count: 30.0 support: 42.0%  s2_r count: 0.0 support: 0.0% ,
   cit2018 > 163729.500
  s1_r count: 25.0 support: 35.0%  s2_r count: 0.0 support: 0.0% ,
  h5Index > 175.500 AND pub2016 > 8230.000
  s1_r count: 31.0 support: 44.0%  s2_r count: 0.0 support: 0.0% ,
   cit2018 > 163729.500
  s1_r count: 25.0 support: 35.0%  s2_r count: 0.0 support: 0.0% ,
   cit2018 > 163729.500
  s1_r count: 25.0 support: 35.0%  s2_r count: 0.0 support: 0.0% ,
  h5Index > 182.000 AND pub2016 > 8230.000
  s1_r count: 30.0 support: 42.0%  s2_r count: 0.0 support: 0.0% ,
   cit2018 > 129332.500 AND pubPA2016 > 0.964
  s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
  pub2016 > 6828.000 AND  cit2018 > 163729.500
  s1_r count: 25.0 support: 35.0%  s2_r count: 0.0 support: 0.0% ,
   cit2018 > 129332.500 AND pubPA2016 > 0.964
  s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
   cit2018 > 163729.500
  s1_r count: 25.0 support: 35.0%

### Selection algorithm results

In [ ]:
get_patterns_SelectionAlg('Universities_RQS_Update.csv', t_count = 10000)

[[pub2017 > 7144.000 AND  cit2017 > 201201.500
  s1_r count: 27.0 support: 38.0%  s2_r count: 0.0 support: 0.0% ,
   cit2019 > 93347.000 AND pub2016 <= 14325.500
  s1_r count: 28.0 support: 39.0%  s2_r count: 0 support: 0.0% ,
   cit2019 > 93347.000 AND h5Index > 175.500
  s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
   cit2019 > 103623.500
  s1_r count: 28.0 support: 39.0%  s2_r count: 0.0 support: 0.0% ,
   cit2019 > 93347.000 AND h5Index > 175.500
  s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
   cit2019 > 93347.000 AND pub2016 <= 14325.500
  s1_r count: 28.0 support: 39.0%  s2_r count: 0 support: 0.0% ,
   cit2019 > 93347.000 AND pub2016 <= 14325.500
  s1_r count: 28.0 support: 39.0%  s2_r count: 0 support: 0.0% ,
  pub2017 > 7144.000 AND  cit2017 > 201201.500
  s1_r count: 27.0 support: 38.0%  s2_r count: 0.0 support: 0.0% ,
   cit2019 > 93347.000 AND h5Index > 175.500
  s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
   cit

### Max Depth results

In [ ]:
# MaxDepth = 2
get_patterns_Max_depth('Universities_RQS_Update.csv', t_count=10000,maxdepth=2)

[pubTJP2019 > 42.550
 s1_r count: 46.0 support: 65.0%  s2_r count: 14.0 support: 20.0% ,
 pubTCP2018 > 19.950
 s1_r count: 36.0 support: 51.0%  s2_r count: 12.0 support: 17.0% ,
  cit2019 > 93347.000
 s1_r count: 33.0 support: 46.0%  s2_r count: 1.0 support: 1.0% ,
  cit2020 > 58036.500
 s1_r count: 34.0 support: 48.0%  s2_r count: 1.0 support: 1.0% ,
 authors2020 > 7976.000
 s1_r count: 38.0 support: 54.0%  s2_r count: 4.0 support: 6.0% ,
  cit2018 > 129332.500
 s1_r count: 37.0 support: 52.0%  s2_r count: 3.0 support: 4.0% ,
  cit2020 > 58036.500
 s1_r count: 34.0 support: 48.0%  s2_r count: 1.0 support: 1.0% ,
  cit2019 > 93347.000
 s1_r count: 33.0 support: 46.0%  s2_r count: 1.0 support: 1.0% ,
 pubTJP2016 > 46.350
 s1_r count: 36.0 support: 51.0%  s2_r count: 14.0 support: 20.0% ,
 authors2017 > 7051.000
 s1_r count: 38.0 support: 54.0%  s2_r count: 4.0 support: 6.0% ,
  cit2017 > 177737.000
 s1_r count: 35.0 support: 49.0%  s2_r count: 2.0 support: 3.0% ,
 h5Index > 182.000
 s1_

In [ ]:
# MaxDepth = 3
get_patterns_Max_depth('Universities_RQS_Update.csv', t_count=10000,maxdepth=3)

[citPP2016 > 23.050 AND pub2018 > 7165.000
 s1_r count: 34.0 support: 48.0%  s2_r count: 2.0 support: 3.0% ,
 authors2016 > 6512.500 AND h5Index > 175.500
 s1_r count: 37.0 support: 52.0%  s2_r count: 1.0 support: 1.0% ,
 pub2019 > 7327.500 AND citPP2020 > 6.050
 s1_r count: 28.0 support: 39.0%  s2_r count: 0.0 support: 0.0% ,
  cit2016 > 229813.000
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 authors2020 > 7976.000 AND pubTCP2020 > 16.300
 s1_r count: 27.0 support: 38.0%  s2_r count: 0.0 support: 0.0% ,
  cit2016 > 229813.000
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 pub2017 > 7144.000 AND pubTCP2017 > 19.200
 s1_r count: 24.0 support: 34.0%  s2_r count: 0.0 support: 0.0% ,
 authors2020 > 7976.000 AND acaCol2016 > 5.250
 s1_r count: 28.0 support: 39.0%  s2_r count: 0.0 support: 0.0% ,
  cit2017 > 177737.000 AND pubPA2019 > 0.961
 s1_r count: 31.0 support: 44.0%  s2_r count: 0.0 support: 0.0% ,
 authors2016 > 6512.500 AND  cit2017 > 201201

In [ ]:
# MaxDepth = 4
get_patterns_Max_depth('Universities_RQS_Update.csv', t_count=10000,maxdepth=4)

[ cit2018 > 129332.500 AND  cit2016 > 229813.000
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
  cit2016 > 229813.000
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 authors2019 > 7306.500 AND  cit2018 > 163729.500
 s1_r count: 25.0 support: 35.0%  s2_r count: 0.0 support: 0.0% ,
  cit2019 > 93347.000 AND citPA2020 > 3.236
 s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
  cit2019 > 93347.000 AND citPP2018 > 11.450
 s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
 pub2018 > 7280.500 AND  cit2017 > 201201.500
 s1_r count: 27.0 support: 38.0%  s2_r count: 0.0 support: 0.0% ,
  cit2017 > 177737.000 AND citPA2020 > 5.842
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
  cit2019 > 93347.000 AND fwCitImp2019 > 1.175
 s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
 authors2016 > 6512.500 AND  cit2016 > 229813.000
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 author

In [ ]:
# Maxdepth = 5
get_patterns_Max_depth('Universities_RQS_Update.csv', t_count=10000,maxdepth=5)

[h5Index > 182.000 AND authors2020 > 7826.500 AND pubPA2020 > 0.948
 s1_r count: 32.0 support: 45.0%  s2_r count: 0.0 support: 0.0% ,
  cit2019 > 93347.000 AND pubTJP2017 > 29.200
 s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
 pub2016 > 6828.000 AND fwCitImp2020 > 1.740
 s1_r count: 28.0 support: 39.0%  s2_r count: 0.0 support: 0.0% ,
  cit2017 > 177737.000 AND pubPA2016 > 0.951
 s1_r count: 32.0 support: 45.0%  s2_r count: 0.0 support: 0.0% ,
  cit2018 > 129332.500 AND acaCol2016 > 5.350
 s1_r count: 26.0 support: 37.0%  s2_r count: 0.0 support: 0.0% ,
 authors2019 > 7306.500 AND h5Index > 175.500 AND  cit2017 > 195964.500
 s1_r count: 31.0 support: 44.0%  s2_r count: 0.0 support: 0.0% ,
 pub2016 > 6828.000 AND citPA2016 > 26.821
 s1_r count: 27.0 support: 38.0%  s2_r count: 0.0 support: 0.0% ,
 authors2016 > 6512.500 AND  cit2019 > 103623.500
 s1_r count: 28.0 support: 39.0%  s2_r count: 0.0 support: 0.0% ,
  cit2020 > 58036.500 AND pub2020 > 8167.000
 s1_r count:

### Min Object by leaf

In [ ]:
# MinObj = 3
get_patterns_Min_Obj('Universities_RQS_Update.csv', t_count=10000, min_obj_by_leaf=3)

[authors2020 > 7976.000 AND acaCol2016 > 5.250
 s1_r count: 28.0 support: 39.0%  s2_r count: 0.0 support: 0.0% ,
 pub2016 > 6828.000 AND h5Index > 175.500 AND  cit2016 > 229813.000
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 authors2019 > 7306.500 AND  cit2020 > 58742.000
 s1_r count: 31.0 support: 44.0%  s2_r count: 0.0 support: 0.0% ,
 pub2019 > 7327.500 AND h5Index > 175.500 AND pubPA2016 > 0.951
 s1_r count: 34.0 support: 48.0%  s2_r count: 0.0 support: 0.0% ,
 pub2020 > 6831.500 AND citPA2018 > 18.798 AND citPA2019 > 12.230
 s1_r count: 22.0 support: 31.0%  s2_r count: 0.0 support: 0.0% ,
 pub2017 > 7144.000 AND citPA2016 > 26.821
 s1_r count: 28.0 support: 39.0%  s2_r count: 0.0 support: 0.0% ,
 authors2020 > 7976.000 AND h5Index > 175.500 AND pub2017 > 8481.000
 s1_r count: 32.0 support: 45.0%  s2_r count: 0.0 support: 0.0% ,
 authors2017 > 7051.000 AND acaCol2017 > 5.300
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 authors2019 > 7306

In [ ]:
# MinObj = 4
get_patterns_Min_Obj('Universities_RQS_Update.csv', t_count=10000, min_obj_by_leaf=4)

[ cit2019 > 93347.000 AND acaCol2018 > 2.650
 s1_r count: 33.0 support: 46.0%  s2_r count: 0.0 support: 0.0% ,
 pub2020 > 6831.500 AND authors2016 > 8866.000 AND citPP2018 > 11.450
 s1_r count: 25.0 support: 35.0%  s2_r count: 0.0 support: 0.0% ,
  cit2017 > 177737.000 AND h5Index > 193.500
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
 pub2018 > 7280.500 AND citPA2020 > 6.179
 s1_r count: 29.0 support: 41.0%  s2_r count: 0.0 support: 0.0% ,
  cit2017 > 177737.000 AND pubPA2018 > 0.984
 s1_r count: 31.0 support: 44.0%  s2_r count: 0.0 support: 0.0% ,
  cit2017 > 177737.000 AND  cit2020 > 58036.500
 s1_r count: 31.0 support: 44.0%  s2_r count: 0.0 support: 0.0% ,
 authors2017 > 7051.000 AND citPP2018 > 13.450 AND pub2018 > 8905.000
 s1_r count: 28.0 support: 39.0%  s2_r count: 0.0 support: 0.0% ,
  cit2020 > 58036.500 AND  cit2018 > 128133.000
 s1_r count: 32.0 support: 45.0%  s2_r count: 0.0 support: 0.0% ,
 pub2020 > 6831.500 AND pub2017 > 9344.500 AND acaCol2019 >

In [ ]:
# MinObj = 5
get_patterns_Min_Obj('Universities_RQS_Update.csv', t_count=10000, min_obj_by_leaf=5)

In [ ]:
# MinObj = 10
get_patterns_Min_Obj('Universities_RQS_Update.csv', t_count=10000, min_obj_by_leaf=10)

## Pytest

** universities.csv file is required for the test functions

In [ ]:
pytest.main(args=['-v'])

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-6.2.5, py-1.10.0, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: jupyter-pytest-2-1.0.1, typeguard-2.7.1
collecting ... collected 23 items

::test_load_train_test_data PASSED                                       [  4%]
::test_get_best_patterns PASSED                                          [  8%]
::test_get_patterns_PBC4cip_classifier PASSED                            [ 13%]
::test_rand_feature_selection PASSED                                     [ 17%]
::test_random_feature_dataframe_generation PASSED                        [ 21%]
::test_get_random_feature_dataframe PASSED                               [ 26%]
::test_algorithm_feature_selection PASSED                                [ 30%]
::test_ETC_attribute_selection PASSED                                    [ 34%]
::test_fit_selector PASSED                                    

<ExitCode.OK: 0>